In [155]:
import pandas as pd
import re
import unicodedata
import os
import sys
sys.path.append(os.path.abspath(".."))
import packages.Preprocesamiento as ppr

In [156]:
df = ppr.cargar_objeto(nombre='comentarios_hoteles', origen='trf')
df = pd.DataFrame(df)

In [157]:
df_nlp = df[[
    "hotel",
    "fecha",
    "titulo",
    "comentario positivo",
    "comentario negativo",
    "nota",
    "tags"]].copy()

df_nlp["nota"] = pd.to_numeric(df_nlp["nota"], errors="coerce")

In [158]:
df_nlp["comentarios"] = (
    df["titulo"].fillna("") + " " +
    df["comentario positivo"].fillna("") + " " +
    df["comentario negativo"].fillna(""))

In [ ]:
stopwords = {
    "el","la","los","las","de","del","y","o","en","con","para",
    "un","una","unos","unas","que","es","por","al","lo", "estancia", "apartamento",
    "dormitorio", "enviado", "movil", "ninos", "pequenos", "dormitorios", "noche", "noches"}

def limpiar_texto(texto):
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')  # quita acentos
    texto = re.sub(r'[^a-zñ\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = " ".join(w for w in texto.split() if w not in stopwords)
    return texto

df_nlp["comentarios_limpios"] = df_nlp["comentarios"].apply(limpiar_texto)
df_nlp["tags_limpios"] = df_nlp["tags"].apply(limpiar_texto)

In [160]:
df_nlp["sentimiento"] = pd.cut(
    df_nlp["nota"],
    bins=[0, 4, 6, 10],
    labels=["negativo", "neutral", "positivo"])

In [161]:
df_nlp_final = df_nlp[["hotel", "fecha", "comentarios_limpios", "sentimiento", "nota", 'tags_limpios']]

In [162]:
ppr.guardar_objeto(obj=df_nlp_final, nombre='comentarios_hoteles_nlp_limpios', destino='trf')